In [1]:
#Import libraires
import spacy
import gensim
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim

In [2]:
#Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Avi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#NLTK Stop Words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'sxsw'])

In [34]:
#Import Dataset
df = pd.read_csv('train.csv')
print(df.head())

   tweet_id                                              tweet  sentiment
0      1701  #sxswnui #sxsw #apple defining language of tou...          1
1      1851  Learning ab Google doodles! All doodles should...          1
2      2689  one of the most in-your-face ex. of stealing t...          2
3      4525  This iPhone #SXSW app would b pretty awesome i...          0
4      3604  Line outside the Apple store in Austin waiting...          1


In [35]:
#Convert to string
df['tweet'] = df['tweet'].astype('str')

In [36]:
#Change encoding of df['tweet']
df['tweet'] = df['tweet'].apply(lambda x: x.encode('ascii', 'ignore'))
df['tweet'] = df['tweet'].apply(lambda x: x.decode('utf-8'))

In [39]:
# function to remove user handles
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for match in r:
        input_txt = re.sub(match, '', input_txt)
    return input_txt

df['clean_tweets'] = df['tweet'].apply(lambda row: remove_pattern(row, "@[\w]*"))
df['clean_tweets'] = df['clean_tweets'].apply(lambda row: remove_pattern(row, '\B#\w*[a-zA-Z]+\w*'))

<>:8: DeprecationWarning: invalid escape sequence \w
<>:9: DeprecationWarning: invalid escape sequence \B
<>:8: DeprecationWarning: invalid escape sequence \w
<>:9: DeprecationWarning: invalid escape sequence \B
<>:8: DeprecationWarning: invalid escape sequence \w
<>:9: DeprecationWarning: invalid escape sequence \B
<ipython-input-39-8338d82a4675>:8: DeprecationWarning: invalid escape sequence \w
  df['clean_tweets'] = df['tweet'].apply(lambda row: remove_pattern(row, "@[\w]*"))
<ipython-input-39-8338d82a4675>:9: DeprecationWarning: invalid escape sequence \B
  df['clean_tweets'] = df['clean_tweets'].apply(lambda row: remove_pattern(row, '\B#\w*[a-zA-Z]+\w*'))


In [42]:
tweets = df['clean_tweets'].tolist()
#Tokenization and preprocess of words
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))

tweet_words = list(sent_to_words(tweets))
print(tweet_words[:1])

[['defining', 'language', 'of', 'touch', 'with', 'different', 'dialects', 'becoming', 'smaller']]


In [44]:
#Build bigram and trigram models
bigram = gensim.models.Phrases(tweet_words, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[tweet_words], threshold = 100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[tweet_words[0]]])

['defining', 'language', 'of', 'touch', 'with', 'different', 'dialects', 'becoming', 'smaller']


In [45]:
#Define fucntions for stopwords, bigrams, trigrams, lemmatization
def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
  return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
  return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']):
  texts_outs = []
  for sent in texts:
    doc = nlp(' '.join(sent))
    texts_outs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return texts_outs

In [46]:
#Remove Stopwords
data_words_nostops = remove_stopwords(tweet_words)

#Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['define', 'language', 'touch', 'different', 'dialect', 'become', 'small']]


In [47]:
#Create dictionary
id2word = corpora.Dictionary(data_lemmatized)

#Create Corpus
texts = data_lemmatized

#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [48]:
#Build LDA model
lda_model = gensim.models.LdaModel(corpus = corpus,
                                   id2word = id2word,
                                   num_topics = 10,
                                   random_state = 100,
                                   update_every = 1,
                                   chunksize = 100,
                                   passes = 10,
                                   alpha = 'auto',
                                   per_word_topics = True)

In [49]:
#Print the keywords in 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.094*"app" + 0.069*"win" + 0.055*"downtown" + 0.035*"phone" + 0.031*"hear" '
  '+ 0.029*"sell" + 0.025*"live" + 0.024*"could" + 0.019*"also" + '
  '0.017*"news"'),
 (1,
  '0.303*"link" + 0.128*"ipad" + 0.115*"store" + 0.076*"apple" + 0.052*"pop" + '
  '0.037*"open" + 0.027*"launch" + 0.020*"temporary" + 0.013*"amp" + '
  '0.012*"first"'),
 (2,
  '0.155*"quot" + 0.072*"come" + 0.048*"need" + 0.035*"give" + 0.033*"show" + '
  '0.026*"buy" + 0.023*"still" + 0.022*"cool" + 0.021*"case" + 0.015*"try"'),
 (3,
  '0.108*"get" + 0.085*"see" + 0.040*"good" + 0.026*"build" + 0.025*"love" + '
  '0.025*"tonight" + 0.023*"let" + 0.022*"head" + 0.021*"detail" + '
  '0.021*"well"'),
 (4,
  '0.097*"go" + 0.051*"make" + 0.038*"want" + 0.033*"take" + 0.032*"party" + '
  '0.030*"day" + 0.023*"video" + 0.020*"user" + 0.020*"event" + 0.019*"next"'),
 (5,
  '0.057*"mobile" + 0.044*"set" + 0.041*"shop" + 0.039*"great" + '
  '0.033*"design" + 0.029*"session" + 0.029*"week" + 0.023*"map" + '
  '0.023*"

In [50]:
#Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.421837 -0.139520       1        1  25.609249
4      0.055490 -0.115059       2        1  10.610774
2      0.057948 -0.037280       3        1   9.544953
8      0.064213 -0.018239       4        1   9.129322
9      0.064045 -0.017716       5        1   8.976543
7     -0.113106  0.386919       6        1   8.344279
5      0.074839 -0.018919       7        1   7.911160
3      0.065087 -0.018226       8        1   7.827444
0      0.067732 -0.013460       9        1   7.573921
6      0.085589 -0.008500      10        1   4.472355, topic_info=     Category         Freq            Term        Total  loglift  logprob
17    Default  3302.000000            link  3302.000000  30.0000  30.0000
29    Default  1393.000000            ipad  1393.000000  29.0000  29.0000
32    Default  1252.000000           store  1252.000000  28.0000  28.0000
18    Default   631.000000            quot   631.000000  27.0000  27.0000
31    Default   590.000000             new   590.000000  26.0000  26.0000
34    Default   824.000000           apple   824.000000  25.0000  25.0000
89    Default   416.000000          social   416.000000  24.0000  24.0000
44    Default   437.000000              go   437.000000  23.0000  23.0000
30    Default   407.000000            line   407.000000  22.0000  22.0000
85    Default   389.000000          circle   389.000000  21.0000  21.0000
90    Default   385.000000           today   385.000000  20.0000  20.0000
207   Default   360.000000             get   360.000000  19.0000  19.0000
232   Default   302.000000             app   302.000000  18.0000  18.0000
79    Default   571.000000             pop   571.000000  17.0000  17.0000
46    Default   283.000000             see   283.000000  16.0000  16.0000
55    Default   292.000000            come   292.000000  15.0000  15.0000
40    Default   239.000000            free   239.000000  14.0000  14.0000
175   Default   221.000000             win   221.000000  13.0000  13.0000
78    Default   399.000000            open   399.000000  12.0000  12.0000
201   Default   214.000000            time   214.000000  11.0000  11.0000
57    Default   230.000000            make   230.000000  10.0000  10.0000
86    Default   400.000000          launch   400.000000   9.0000   9.0000
172   Default   192.000000          mobile   192.000000   8.0000   8.0000
143   Default   198.000000          people   198.000000   7.0000   7.0000
114   Default   195.000000            need   195.000000   6.0000   6.0000
155   Default   177.000000        downtown   177.000000   5.0000   5.0000
124   Default   187.000000           check   187.000000   4.0000   4.0000
88    Default   178.000000        possibly   178.000000   3.0000   3.0000
87    Default   178.000000           major   178.000000   2.0000   2.0000
98    Default   168.000000            know   168.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
384   Topic10    87.716858            talk    88.598099   3.0973  -3.0774
137   Topic10    81.209167            find    82.090401   3.0965  -3.1545
138   Topic10    68.816315            game    69.697556   3.0945  -3.3200
389   Topic10    63.444069         twitter    64.325432   3.0935  -3.4013
64    Topic10    56.921146          future    57.802422   3.0919  -3.5098
62    Topic10    56.021000           watch    56.902306   3.0916  -3.5258
379   Topic10    51.482403            room    52.363823   3.0903  -3.6102
440   Topic10    48.407822           share    49.289082   3.0892  -3.6718
702   Topic10    40.926395          boomer    41.807667   3.0860  -3.8397
465   Topic10    39.012779             yet    39.894135   3.0849  -3.8876
512   Topic10    38.375095            spot    39.256466   3.0845  -3.9041
330   Topic10    37.187508            full    38.068855   3.0838  -3.9355
331   Topic10    34.746803    presentation    35.6281

In [53]:
#Find the dominant topic in each document
def format_topic_sentences(ldamodel = None, corpus = corpus, texts = tweets):
    sent_topics_df = pd.DataFrame()
    
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key = lambda x: x[1], reverse = True)
        for j ,(topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ','.join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index = True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis = 1)
    return sent_topics_df

df_topics_sents_keywords = format_topic_sentences(ldamodel = lda_model, corpus = corpus, texts = tweets)

In [54]:
#Set column names and view dataset
df_topics_sents_keywords.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Text']
df_topics_sents_keywords.head(10)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
0,4.0,0.4140,"go,make,want,take,party,day,video,user,event,next",defining language of touch with different d...
1,1.0,0.5423,"link,ipad,store,apple,pop,open,launch,temporar...",Learning ab Google doodles! All doodles should...
2,2.0,0.4511,"quot,come,need,give,show,buy,still,cool,case,try",one of the most in-your-face ex. of stealing t...
3,8.0,0.3156,"line,check,look,wait,use,would,android,may,pla...",This iPhone app would b pretty awesome if it ...
4,1.0,0.3645,"link,ipad,store,apple,pop,open,launch,temporar...",Line outside the Apple store in Austin waiting...
5,1.0,0.5946,"link,ipad,store,apple,pop,open,launch,temporar...",One lone dude awaits iPad 2 at Apples SXSW st...
6,1.0,0.3106,"link,ipad,store,apple,pop,open,launch,temporar...","SXSW Tips, Prince, NPR Videos, Toy Shopping Wi..."
7,1.0,0.3102,"link,ipad,store,apple,pop,open,launch,temporar...",NU user RT New for now in the App Store inc...
8,1.0,0.2584,"link,ipad,store,apple,pop,open,launch,temporar...",Free sampler on iTunes {link}
9,1.0,0.2063,"link,ipad,store,apple,pop,open,launch,temporar...",I think I might go all weekend without seeing ...


In [55]:
#Most representive sentence for each topic
pd.options.display.max_colwidth = 100
sent_topics_df2 = pd.DataFrame()
sent_topics_grped = df_topics_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_grped:
    sent_topics_df2 = pd.concat([sent_topics_df2, grp.sort_values(['Perc_Contribution'], ascending = False).head(1)],axis = 0)
    
sent_topics_df2.reset_index(drop=True, inplace = True)

sent_topics_df2.columns = ['Topic_Num', 'Topic_Perc_Contribution', 'Keywords', 'Representative_Text']

sent_topics_df2.head(10)

,Topic_Num,Topic_Perc_Contribution,Keywords,Representative_Text
0,0.0,0.4328,"app,win,downtown,phone,hear,sell,live,could,also,news",Atomic Tom rocked at the CNN Grill Party. They're from Brooklyn. Google them. Also: the food and...
1,1.0,0.6314,"link,ipad,store,apple,pop,open,launch,temporary,amp,first","We're giving away a shiny iPad 2 at SXSW Quotables, simply sign up to be entered automatically! ..."
2,2.0,0.5023,"quot,come,need,give,show,buy,still,cool,case,try",recounts the batshit insanity doubling as &quot;creativity&quot; that ensures they &quot;don't ...
3,3.0,0.4817,"get,see,good,build,love,tonight,let,head,detail,well",Yup. RT I woke up before my iPhone finished recharging. That means I got enough sleep last nigh...
4,4.0,0.5197,"go,make,want,take,party,day,video,user,event,next","The only way you truly understand a material is by making things with it. Jonathan Ive , Apple ..."
5,5.0,0.4429,"mobile,set,shop,great,design,session,week,map,really,search",Brought up how Google Maps had rerouted all images of JCPenney to images of Macys or trashy rest...
6,6.0,0.5103,"talk,find,game,twitter,future,watch,room,share,boomer,yet",RT Best iPad design: Big chunky elements. Generous space. Clarity trumps density. Tap quality t...
7,7.0,0.5343,"new,social,circle,today,possibly,major,big,launch,product,network","Google to Launch Major New Social Network Called Circles, Possibly Today {link} of course, anot..."
8,8.0,0.5116,"line,check,look,wait,use,would,android,may,play,interesting",Google maps used to provide common reference points and printed for fast paced on the ground rea...
9,9.0,0.4841,"free,time,people,know,year,iphone,thank,work,think,last",If you thought people spent a lot of time staring at their iPhone last year at ...just wait.


In [56]:
#Topic Distribution across documents
topic_counts = df_topics_sents_keywords['Dominant_Topic'].value_counts()

topic_contribution = round(topic_counts/sum(topic_counts), 4)

topic_num_keywords = df_topics_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis = 1)

df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

df_dominant_topics.head(10)

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,4.0,"go,make,want,take,party,day,video,user,event,next",178.0,0.0245
1,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",4718.0,0.6486
2,2.0,"quot,come,need,give,show,buy,still,cool,case,try",392.0,0.0539
3,8.0,"line,check,look,wait,use,would,android,may,play,interesting",191.0,0.0263
4,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",424.0,0.0583
5,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",291.0,0.0400
6,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",128.0,0.0176
7,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",411.0,0.0565
8,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",238.0,0.0327
9,1.0,"link,ipad,store,apple,pop,open,launch,temporary,amp,first",303.0,0.0417
